# Generate the training data (wall + climber + pos)
- Wall: 15 different walls
- Climber: 300 climbers divided into casual, skilled, elite
- Pos: current position (hands position and feet position)

In [53]:
import pandas as pd

# Load the files
walls = pd.read_csv("climbing_data.csv")
climbers = pd.read_csv("all_climbers.csv")

In [54]:
climbers = climbers.drop(columns=["Unnamed: 0"])

In [55]:
from ast import literal_eval
walls["points"] = walls["points"].apply(literal_eval)

In [56]:
import sys
sys.path.append("../src")

from dataset_function import sample_grap_points_by_radius
# Generate the Training Data
times = 20
data = {
    "route": [],
    "climber": [],
    "hands": [],
    "feet": []
}

for w in walls["points"]:
    for _, c in climbers.iterrows():
        for n in range(times):
            hands, feet, center = sample_grap_points_by_radius(route=w,
                                                               climber=c,
                                                               max_trials=100)
            data["route"].append(w)
            data["climber"].append({"group": c["group"],
                                    "height": c["height"],
                                    "weight": c["weight"],
                                    "ape_index": c["ape_index"],
                                    "strength": c["strength"],
                                    "flexibility": c["flexibility"],
                                    "leg_len_factor": c["leg_len_factor"]})
            data["hands"].append(hands)
            data["feet"].append(feet)

In [57]:
df = pd.DataFrame(data)

In [ ]:
df.to_csv("reachability_dataset.csv", index=False)

# Generate the training data (wall + climber + wall features + pos) (for Reachability Model with Holds features)

In [1]:
import pandas as pd
import numpy as np
from itertools import product
from sklearn.neighbors import NearestNeighbors
import torch
from torch_geometric.data import Data
import sys
sys.path.append("../src")
from dataset_function import sample_hand_foot_points
from dataset_function import get_reachability_label_with_shape

In [2]:
walls_with_features = pd.read_csv("climbing_with_shape_features.csv")
climbers = pd.read_csv("all_climbers.csv").drop(columns=["Unnamed: 0"])

In [3]:
climbers

,group,height,weight,ape_index,strength,flexibility,leg_len_factor
0,casual,163.3,73.8,1.00,65,5,0.510348
1,casual,162.4,62.6,1.02,67,4,0.472300
2,casual,169.0,50.8,1.02,62,4,0.498026
3,casual,158.3,33.0,0.95,74,5,0.601822
4,casual,163.7,91.1,1.03,67,5,0.520042
...,...,...,...,...,...,...,...
295,elite,160.8,64.4,1.07,99,9,0.500846
296,elite,184.2,67.8,1.07,105,8,0.578098
297,elite,163.3,69.4,1.09,95,9,0.570675
298,elite,167.1,72.2,1.01,96,9,0.601026


In [4]:
unique_walls = walls_with_features["filename"].unique()
wall_climber_pairs = [
    (filename, idx, climbers.loc[idx, "group"])
    for filename, idx in product(unique_walls, climbers.index)
]

In [16]:
graph_list = []
times = 20

for filename, group in walls_with_features.groupby("filename"):
    group = group.dropna(subset=["center_x", "center_y"])
    # Filter out the holds of wall that less than 6
    if len(group) < 6:
        continue

    route_points = group.to_dict(orient='records')

    for _, climber in climbers.iterrows():
        for n in range(times):
            hands, feet = sample_hand_foot_points(route_points, climber)
            if hands is None or feet is None:
                continue
        
            node_feats = []
            labels = []

            for p in route_points:
                node_feats.append([
                    p["center_x"],
                    p["center_y"],
                    p.get("shape_area", 0),
                    p.get("shape_perimeter"),
                    p.get("shape_aspect_ratio", 0),
                    p.get("shape_circularity", 0)
                ])
                labels.append(get_reachability_label_with_shape(p, hands, feet, climber))
            
            label_array = np.array(labels)
            zero_ratio = np.mean(label_array == 0)
            if zero_ratio > 0.95:
                continue

            x = torch.tensor(node_feats, dtype=torch.float)
            y = torch.tensor(labels, dtype=torch.long)

            # Edge construction via KNN
            pos = np.array([[p["center_x"], p["center_y"]] for p in route_points])
            nbrs = NearestNeighbors(n_neighbors=min(8, len(pos))).fit(pos)
            _, indices = nbrs.kneighbors(pos)

            edge_index = []
            for i, neigh in enumerate(indices):
                for j in neigh:
                    if i != j:
                        edge_index.append([i, j])
            edge_index = torch.tensor(edge_index, dtype=torch.long).T

            climber_feat = torch.tensor([
                climber["height"], climber["ape_index"],
                climber["flexibility"], climber["leg_len_factor"]
            ], dtype=torch.float)

            graph = Data(x=x, edge_index=edge_index, y=y, climber=climber_feat)
            graph_list.append(graph)

In [17]:
len(graph_list)

89644

In [19]:
torch.save(graph_list, "graph_list_holds_features.pt")